# Prepare the dataset

In [1]:
import urllib.request
import zipfile

In [2]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"

In [3]:
file_name = "horse-or-human.zip"
training_dir = "horse-or-human/training/"

In [4]:
urllib.request.urlretrieve(url,file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x7fb6bb539e80>)

In [5]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()


# Processing 

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )
train_generator = train_datagen.flow_from_directory(
      training_dir,
      target_size=(300, 300),
      class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [8]:
validation_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [9]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
      validation_dir,
      target_size=(300, 300),
      class_mode='binary'
)


Found 256 images belonging to 2 classes.


In [10]:
model = tensorflow.keras.models.Sequential([
      tensorflow.keras.layers.Conv2D(16, (3,3), activation='relu' ,
                  input_shape=(300, 300, 3)),
      tensorflow.keras.layers.MaxPooling2D(2, 2),
      tensorflow.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Flatten(),
      tensorflow.keras.layers.Dense(512, activation='relu'),
      tensorflow.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:

model.compile(loss='binary_crossentropy',
           optimizer=tensorflow.keras.optimizers.RMSprop(learning_rate=0.001),
metrics=['accuracy'])

In [12]:
history = model.fit(
      train_generator,
      epochs=15,
      validation_data=validation_generator,
)

Epoch 1/15
33/33 [==============================] - 116s 3s/step - loss: 0.7168 - accuracy: 0.5833 - val_loss: 2.8915 - val_accuracy: 0.5000
Epoch 2/15
33/33 [==============================] - 112s 3s/step - loss: 0.6451 - accuracy: 0.6981 - val_loss: 0.7479 - val_accuracy: 0.5195
Epoch 3/15
33/33 [==============================] - 111s 3s/step - loss: 0.4790 - accuracy: 0.7605 - val_loss: 0.5542 - val_accuracy: 0.7109
Epoch 4/15
33/33 [==============================] - 112s 3s/step - loss: 0.4401 - accuracy: 0.8004 - val_loss: 0.5753 - val_accuracy: 0.6484
Epoch 5/15
33/33 [==============================] - 112s 3s/step - loss: 0.3732 - accuracy: 0.8335 - val_loss: 1.4867 - val_accuracy: 0.5820
Epoch 6/15
33/33 [==============================] - 111s 3s/step - loss: 0.3099 - accuracy: 0.8734 - val_loss: 4.1473 - val_accuracy: 0.5273
Epoch 7/15
33/33 [==============================] - 112s 3s/step - loss: 0.2777 - accuracy: 0.9007 - val_loss: 1.2347 - val_accuracy: 0.7773
Epoch 8/15
33

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

### Testing Horse or Human

In [14]:
import numpy
from google.colab import files
from keras.preprocessing import image 
from tensorflow.keras.utils import load_img , img_to_array

In [15]:
from keras.utils.image_dataset import image_utils
uploaded = files.upload()
for fn in uploaded.keys():
  # predicting images
  path='/content/'+fn
  img = load_img(path,target_size=(300,300))
  x = img_to_array(img)
  x = numpy.expand_dims(x,axis=0)
  image_tensor = numpy.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn+ "is a human")
  else:
    print(fn+" is a horse")


Saving horse-2063672__340.jpg to horse-2063672__340.jpg
Saving woman-pic.webp to woman-pic.webp
1/1 [==============================] - 0s 189ms/step
[[3.668072e-05]]
[3.668072e-05]
horse-2063672__340.jpg is a horse
1/1 [==============================] - 0s 39ms/step
[[1.]]
[1.]
woman-pic.webpis a human
